# Voice Recognition using Deep Neural Networks
Gender Recognition by Voice and Speech Analysis

This database was created to identify a voice as male or female, based upon acoustic properties of the voice and speech. The dataset consists of 3,168 recorded voice samples, collected from male and female speakers. The voice samples are pre-processed by acoustic analysis in R using the seewave and tuneR packages, with an analyzed frequency range of 0hz-280hz (human vocal range).

## The Dataset
The following acoustic properties of each voice are measured and included within the CSV:

* meanfreq: mean frequency (in kHz)
* sd: standard deviation of frequency
* median: median frequency (in kHz)
* Q25: first quantile (in kHz)
* Q75: third quantile (in kHz)
* IQR: interquantile range (in kHz)
* skew: skewness (see note in specprop description)
* kurt: kurtosis (see note in specprop description)
* sp.ent: spectral entropy
* sfm: spectral flatness
* mode: mode frequency
* centroid: frequency centroid (see specprop)
* peakf: peak frequency (frequency with highest energy)
* meanfun: average of fundamental frequency measured across acoustic signal
* minfun: minimum fundamental frequency measured across acoustic signal
* maxfun: maximum fundamental frequency measured across acoustic signal
* meandom: average of dominant frequency measured across acoustic signal
* mindom: minimum of dominant frequency measured across acoustic signal
* maxdom: maximum of dominant frequency measured across acoustic signal
* dfrange: range of dominant frequency measured across acoustic signal
* modindx: modulation index. Calculated as the accumulated absolute difference between adjacent measurements of fundamental frequencies divided by the frequency range
* label: male or female

In [7]:
# Step 1: Import Dependencies

import numpy as np
import pandas as pd

In [8]:
# Step 2: Get the Data

voice = pd.read_csv("resources/voice.csv")
voice.head()

,meanfreq,sd,median,Q25,Q75,IQR,skew,kurt,sp.ent,sfm,...,centroid,meanfun,minfun,maxfun,meandom,mindom,maxdom,dfrange,modindx,label
0,0.059781,0.064241,0.032027,0.015071,0.090193,0.075122,12.863462,274.402906,0.893369,0.491918,...,0.059781,0.084279,0.015702,0.275862,0.007812,0.007812,0.007812,0.000000,0.000000,male
1,0.066009,0.067310,0.040229,0.019414,0.092666,0.073252,22.423285,634.613855,0.892193,0.513724,...,0.066009,0.107937,0.015826,0.250000,0.009014,0.007812,0.054688,0.046875,0.052632,male
2,0.077316,0.083829,0.036718,0.008701,0.131908,0.123207,30.757155,1024.927705,0.846389,0.478905,...,0.077316,0.098706,0.015656,0.271186,0.007990,0.007812,0.015625,0.007812,0.046512,male
3,0.151228,0.072111,0.158011,0.096582,0.207955,0.111374,1.232831,4.177296,0.963322,0.727232,...,0.151228,0.088965,0.017798,0.250000,0.201497,0.007812,0.562500,0.554688,0.247119,male
4,0.135120,0.079146,0.124656,0.078720,0.206045,0.127325,1.101174,4.333713,0.971955,0.783568,...,0.135120,0.106398,0.016931,0.266667,0.712812,0.007812,5.484375,5.476562,0.208274,male


### Preprocess the data

Note: The output ("label") is a string and needs to be label-encoded

In [9]:
# Step 3: Assign the X and y

# X is everything but the output (label)
X = voice.drop("label", axis=1)

# y is the output "label"
y = voice['label']

In [10]:
print(X.shape, y.shape)
# X is a 2d vector array and y is a 1d vector array

(3168, 20) (3168,)


In [12]:
# Step 4: split the data into training and testing datasets

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

In [13]:
# Step 5: Scale and fit the data using StandardScaler on the training features, then transform 
# In other words, preprocess the X training set

from sklearn.preprocessing import StandardScaler

X_scaler = StandardScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [15]:
# Step 6: Because our output is a string, we label encode to a readible language for the computer
# Preprocess the output (y)
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

In [16]:
# Step 7: Now that y is label encoded, use One Hot Encoding to make it categorical

from keras.utils import to_categorical

y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

### Create the Deep Learning Neural Network

In [18]:
# Step 8: Create the model using Sequential and Dense
from keras.models import Sequential
from keras.layers import Dense

model = Sequential()
model.add(Dense(units=100, activation='relu', input_dim=20))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=2, activation='softmax'))

In [19]:
# Step 9: Compile the model

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [20]:
# Step 10: Fit (train) the data to the model using the preprocessed training set
model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=100,
    shuffle=True,
    verbose=2
)

Epoch 1/100
75/75 - 0s - loss: 0.2670 - accuracy: 0.9070
Epoch 2/100
75/75 - 0s - loss: 0.0892 - accuracy: 0.9735
Epoch 3/100
75/75 - 0s - loss: 0.0690 - accuracy: 0.9777
Epoch 4/100
75/75 - 0s - loss: 0.0672 - accuracy: 0.9781
Epoch 5/100
75/75 - 0s - loss: 0.0576 - accuracy: 0.9827
Epoch 6/100
75/75 - 0s - loss: 0.0556 - accuracy: 0.9802
Epoch 7/100
75/75 - 0s - loss: 0.0521 - accuracy: 0.9806
Epoch 8/100
75/75 - 0s - loss: 0.0463 - accuracy: 0.9836
Epoch 9/100
75/75 - 0s - loss: 0.0466 - accuracy: 0.9840
Epoch 10/100
75/75 - 0s - loss: 0.0433 - accuracy: 0.9848
Epoch 11/100
75/75 - 0s - loss: 0.0361 - accuracy: 0.9865
Epoch 12/100
75/75 - 0s - loss: 0.0380 - accuracy: 0.9861
Epoch 13/100
75/75 - 0s - loss: 0.0334 - accuracy: 0.9882
Epoch 14/100
75/75 - 0s - loss: 0.0322 - accuracy: 0.9878
Epoch 15/100
75/75 - 0s - loss: 0.0289 - accuracy: 0.9895
Epoch 16/100
75/75 - 0s - loss: 0.0269 - accuracy: 0.9891
Epoch 17/100
75/75 - 0s - loss: 0.0223 - accuracy: 0.9916
Epoch 18/100
75/75 - 0s

In [22]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 100)               2100      
_________________________________________________________________
dense_1 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 202       
Total params: 12,402
Trainable params: 12,402
Non-trainable params: 0
_________________________________________________________________


In [23]:
# Step 11: Quantify the model using the preprocessed test dataset
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2
)

print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

25/25 - 0s - loss: 0.0651 - accuracy: 0.9886
Loss: 0.06507638841867447, Accuracy: 0.9886363744735718


### Make predictions (with 98.9% accuracy)

In [24]:
encoded_predictions = model.predict_classes(X_test_scaled[:5])
prediction_labels = label_encoder.inverse_transform(encoded_predictions)

/Users/gianmillare/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [25]:
print(f"Predicted classes: {prediction_labels}")
print(f"Actual Labels: {list(y_test[:5])}")

Predicted classes: ['female' 'female' 'male' 'female' 'male']
Actual Labels: ['female', 'female', 'male', 'female', 'male']
